In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/README.dataset.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/README.roboflow.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels/construction-3-_mp4-219_jpg.rf.416dbf917b54d5895e2430fa0d6f1b66.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels/construction-1-_mp4-147_jpg.rf.a9062a80b06ac796f7afb269e5f18c53.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels/Movie-on-10-31-22-at-10_08-AM_mov-13_jpg.rf.91dc3b79da617fb2e4321f27f48f21f4.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels/2009_002778_jpg.rf.7c8977ed190a2ce4211ef58f66864066.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/valid/labels/004424_jpg.rf.0470713b945b08839105cde711db62d9.txt
/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/vali

In [2]:
# Step 1: Install necessary libraries
!pip install torch torchvision

# Step 2: Import Libraries
import torch
from torchvision import datasets, transforms, models
import pandas as pd
import yaml
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Step 3: Define Dataset Paths
data_dir = "/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/"  # Root directory of your dataset

train_images = f"{data_dir}/train/images"  # Training images directory
val_images = f"{data_dir}/valid/images"   # Validation images directory
test_images = f"{data_dir}/test/images"   # Test images directory

# Define classes from the summary
classes = [
    "Hardhat", "Mask", "NO-Hardhat", "NO-Mask",
    "NO-Safety Vest", "Person", "Safety Cone",
    "Safety Vest", "machinery", "vehicle"
]

# Step 4: Create Dataset YAML File
dataset_yaml_path = "/kaggle/working/css-data.yaml"
dataset_yaml = {
    "path": data_dir,              # Root directory of the dataset
    "train": train_images,         # Path to training images
    "val": val_images,             # Path to validation images
    "test": test_images,           # Path to test images (optional for YOLOv8)
    "nc": len(classes),            # Number of classes
    "names": classes               # Class names
}

# Save the YAML file
with open(dataset_yaml_path, "w") as file:
    yaml.dump(dataset_yaml, file)
print(f"Dataset YAML file saved to {dataset_yaml_path}")

# Step 5: Load Pre-trained Faster R-CNN Model
# Load a pre-trained Faster R-CNN model with a ResNet backbone
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classification head to match the number of classes in your dataset
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, len(classes))

# Move the model to GPU (if available)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Step 6: Data Transformations for Image Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

# Step 7: Create DataLoader for validation data
from torch.utils.data import Dataset, DataLoader
import cv2

class CustomDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_files = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, img_path

val_dataset = CustomDataset(val_images, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Step 8: Inference and Evaluation
model.eval()  # Set the model to evaluation mode

results = []
for images, image_paths in tqdm(val_loader, desc="Evaluating"):
    images = [image.to(device) for image in images]
    with torch.no_grad():
        predictions = model(images)
    
    # Collecting the results (bounding boxes, labels, scores)
    for i, image_path in enumerate(image_paths):
        pred = predictions[i]
        results.append({
            'image': image_path,
            'boxes': pred['boxes'].cpu().numpy(),
            'labels': pred['labels'].cpu().numpy(),
            'scores': pred['scores'].cpu().numpy()
        })

# Step 9: Extract Validation Metrics (Precision, Recall, mAP)
# For simplicity, we'll print results and assume metrics computation is handled
metrics = {
    "Precision": "calculated_precision_value",  # You can compute precision from `results`
    "Recall": "calculated_recall_value",        # Similarly, compute recall from `results`
    "mAP@0.5": "calculated_map_value"           # Compute mAP from results
}

# Save Metrics to CSV
csv_output_path = "/kaggle/working/validation_results.csv"
df = pd.DataFrame(metrics.items(), columns=["Metric", "Value"])
df.to_csv(csv_output_path, index=False)

print("\nValidation Metrics:")
print(df)



from PIL import Image, ImageDraw

# Step 10: Visualize results (bounding boxes) for an image
def draw_boxes(image_path, boxes, labels, scores, class_names):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    for box, label, score in zip(boxes, labels, scores):
        # Draw bounding box
        draw.rectangle(box.tolist(), outline="red", width=2)
        draw.text((box[0], box[1]), f"{class_names[label]} {score:.2f}", fill="red")
    image.show()

# Visualize results for the first image
draw_boxes(results[0]['image'], results[0]['boxes'], results[0]['labels'], results[0]['scores'], classes)

# Step 11: Print Model Summary
print(model)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.7 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 166MB/s]  
Evaluating: 100%|██████████| 29/29 [08:39<00:00, 17.92s/it]



Validation Metrics:
      Metric                       Value
0  Precision  calculated_precision_value
1     Recall     calculated_recall_value
2    mAP@0.5        calculated_map_value
FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): 